<a href="https://colab.research.google.com/github/Rajat-Kumar-Pandey/MACHINE-LEARNING/blob/main/Text_emotion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text emotion classifier

In [3]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


In [4]:
data = pd.read_csv("/content/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [5]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
sequences = tokenizer.texts_to_sequences(texts)

In [8]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [9]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:

one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [12]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [13]:
one_hot_labels[0]

array([0., 0., 0., 0., 1., 0.])

In [14]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.3698 - loss: 1.5261 - val_accuracy: 0.6753 - val_loss: 0.8937
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.8521 - loss: 0.4664 - val_accuracy: 0.8225 - val_loss: 0.5446
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 0.9825 - loss: 0.0696 - val_accuracy: 0.8400 - val_loss: 0.5551
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 35s 40ms/step - accuracy: 0.9945 - loss: 0.0277 - val_accuracy: 0.8375 - val_loss: 0.5762
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.9959 - loss: 0.0189 - val_accuracy: 0.8363 - val_loss: 0.6103
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.9967 - loss: 0.0146 - val_accuracy: 0.8278 - val_loss: 0.6438
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - accuracy: 0.9970 - loss: 0.0121 - val_accuracy: 0.8288 - val_loss: 0.6854
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.9973 - loss: 0.0109 - val_accurac

In [16]:
input_text = "i don t feel guilty like i m not going to be able to cook for him"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
sadness


In [18]:
import pickle
from keras.models import load_model

def save_model_components(model, tokenizer, label_encoder):
    # Save the trained model in Keras format
    model.save("text_model.keras")

    # Save the tokenizer
    with open("tokenizer.pickle", "wb") as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Save the label encoder
    with open("label_encoder.pickle", "wb") as handle:
        pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("Model, tokenizer, and label encoder saved successfully!")
#


# After training the model, call the save function
save_model_components(model, tokenizer, label_encoder)
print("Model, tokenizer, and label encoder saved successfully!")


Model, tokenizer, and label encoder saved successfully!
Model, tokenizer, and label encoder saved successfully!


In [21]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Check if the file exists
file_path = os.path.join(current_directory, 'text_model.keras')
if os.path.exists(file_path):
    print(f"{file_path} exists.")
else:
    print(f"{file_path} does not exist.")


Current Working Directory: /content
/content/text_model.keras exists.
